## `With`上下文管理器

`with`上下文管理器在Python的处理文件打开与关闭的时候经常使用：

```python
with open('somefile.txt', 'r') as f:
    f.read()
```

`with`可以让代码块在结束时，自动的调用`f.close()`来避免资源的泄漏。

并不是只有`open()`函数返回的对象才能使用`with`语句。实际上，任何对象，只要正确实现了上下文管理，就可以用于`with`语句。

实现上下文管理是通过`__enter__`和`__exit__`这两个方法实现的。例如，下面的class实现了这两个方法：

In [5]:
class Query(object):
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        print('---Enter---')
        return self
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type:
            print('Error')
        else:
            print('---Exit---')
    def query(self):
        print('Query info about: %s...' % self.name)

In [6]:
with Query('hello') as q:
    q.query()

---Enter---
Query info about: hello...
---Exit---


## `contextmanager`

编写`__enter__`和`__exit__`仍然很繁琐，因此Python的标准库`contextlib`提供了更简单的写法，上面的代码可以改写如下：

In [10]:
from contextlib import contextmanager

class Query(object):
    def __init__(self, name):
        self.name = name
    def query(self):
        print('Query info about: %s...' % self.name)
        
@contextmanager
def create_query(name):
    print('---Enter---')
    q = Query(name)
    yield q
    print('---Exit---')

In [12]:
with create_query('hello') as q:
    q.query()

---Enter---
Query info about: hello...
---Exit---


很多时候，我们希望在某段代码执行前后自动执行特定代码，也可以用`@contextmanager`实现。例如：

In [13]:
@contextmanager
def tag(name):
    print("<%s>" % name)
    yield
    print("<%s>" % name)
with tag("h1"):
    print("hello, world")

<h1>
hello, world
<h1>


代码的执行顺序是：

1. 调用tag，打印第一句`print("<%s>" % name)`
2. 遇到tag，当前函数让出执行流，进入到with的代码块内部，执行`print("hello, world")`
3. `with`代码块执行结束后，回来yield后续的代码继续执行`print("<%s>" % name)`。